In [6]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import pandas as pd
from empathetic_dialogues import EmpatheticDialogues
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers.legacy import SGD
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sokan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sokan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [15]:
input_file = "data.json"

# Open the JSON file for reading
with open(input_file, "r") as f:
    # Load the entire JSON data from the file
    json_data = json.load(f)

# Process the loaded JSON data as needed
# For example, print the entire content
print(json_data[0])

{'instruction': 'If you are a licensed psychologist, please provide this patient with a helpful response to their concern.', 'input': "I'm feeling really anxious lately and I don't know why.", 'output': "It's common to feel anxious at times, and there can be many reasons for it. Have there been any recent changes or stressors in your life that may be contributing to your anxiety? Let's work together to identify any triggers and develop coping strategies to manage your anxiety."}


In [11]:
import json
import random
import string

# Input JSONL file
input_file = "data.json"

# Open the JSON file for reading
with open(input_file, "r") as f:
    # Load the entire JSON data from the file
    json_data = json.load(f)
# Output JSON file
output_file = "output.json"

# Transform data to the new format
transformed_data = [
    {
    "tag": f'fact{idx+33}',
        "patterns": [entry["input"]],
        "responses": [entry["output"]]
    }
    for idx,entry in enumerate(json_data)
]

# Write JSON data to the file
with open(output_file, "w") as f:
    json.dump(transformed_data, f, indent=2)


In [12]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
json_data = open('test.json').read()
intents = json.loads(json_data)
print(intents)

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hi there', 'Hello', 'Hey there', 'Howdy', 'Hola', 'Bonjour', 'Konnichiwa', 'Guten tag', 'Ola'], 'responses': ['Hello there. Tell me how are you feeling today?', 'Hi there. What brings you here today?', 'Hi there. How are you feeling today?', 'Great to see you. How do you feel currently?', "Hello there. Glad to see you're back. What's going on in your world right now?"]}, {'tag': 'morning', 'patterns': ['Good morning'], 'responses': ["Good morning. I hope you had a good night's sleep. How are you feeling today? "]}, {'tag': 'afternoon', 'patterns': ['Good afternoon'], 'responses': ['Good afternoon. How is your day going?']}, {'tag': 'evening', 'patterns': ['Good evening'], 'responses': ['Good evening. How has your day been?']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I provide

In [13]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created
model created


C:\Users\sokan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
#model
model = Sequential()
model.add(Dense(2048, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
#histogram = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=10, verbose=1)
#model.save('mental_chatbot_model.h5', histogram)

print("model created")

In [ ]:
#load already trained model
model = load_model('mental_chatbot_model.h5')

In [14]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, histogram):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.01
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probabilit
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [15]:
model = Sequential()
model.add(Dense(2048, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
model.load_weights('mental_chatbot_model.h5')

In [16]:

print(chatbot_response('Hi'))
print(chatbot_response('What is your name'))
print(chatbot_response('can you tell me what mental health'))
print(chatbot_response('my dad died'))
print(chatbot_response('are you dumb'))
print(chatbot_response('I feel like I\'m not good enough'))
print(chatbot_response('I\'m having trouble with my identity and I don\'t know who I am'))
print(chatbot_response('I\'m having trouble with my body image'))
print(chatbot_response('I\'m having trouble coping with the loss of a loved one.'))
print(chatbot_response('I\'m having trouble with my partner'))
print(chatbot_response('i want to hurt myself'))
print(chatbot_response('Where can i seek help about suicide'))

1/1 [==============================] - 0s 125ms/step
Great to see you. How do you feel currently?
1/1 [==============================] - 0s 25ms/step
Im a Medical Assistant Chatbot
1/1 [==============================] - 0s 25ms/step
Sure. Tell me how can i assist you
1/1 [==============================] - 0s 24ms/step
I'm sorry to hear that. If you want to talk about it. I'm here.
1/1 [==============================] - 0s 30ms/step
I wish you wouldn't say such hurtful things. I'm sorry if I wasn't useful
1/1 [==============================] - 0s 25ms/step
It's common to experience self-doubt, but it's important to remember that everyone has strengths and weaknesses. Let's explore what makes you feel this way and work on building self-confidence.
1/1 [==============================] - 0s 25ms/step
Identity exploration is a natural part of life, but it can be challenging to navigate alone. Let's explore your values, beliefs, and experiences to help you better understand your identity. We